# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [59]:
# import libraries

import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [60]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("disaster_messages", con=engine)

# Let's take a quick look if everything looks as expected
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
# Define feature and target variables X and Y
X = df['message']
Y = df.iloc[:, 4:]

In [62]:
# We should have only the category columns here - quick check:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [63]:
def tokenize(text):
    
    """
    Tokenization function.
    """
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens=[]
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [64]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [65]:
# may take a couple of seconds - patience
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [66]:
Y_pred = pipeline.predict(X_test)

In [67]:
def model_testing(Y_test, Y_pred):
    
    """
    Function to iterate through columns and call sklearn classification report on each.
    """
    for index, col in enumerate(Y_test):
        print(col, classification_report(Y_test[col], Y_pred[:, index]))

In [68]:
model_testing(Y_test, Y_pred)

related              precision    recall  f1-score   support

          0       0.63      0.35      0.45      1532
          1       0.83      0.94      0.88      5022

avg / total       0.78      0.80      0.78      6554

request              precision    recall  f1-score   support

          0       0.88      0.98      0.93      5390
          1       0.82      0.39      0.53      1164

avg / total       0.87      0.88      0.86      6554

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6527
          1       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.72      0.87      0.79      3814
          1       0.75      0.52      0.61      2740

avg / total       0.73      0.73      0.72      6554

medical_help              precision    recall  f1-score   support

          0       0.93      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Some labels in Y_test don't appear in Y_pred, which is why a warning message is thrown.

Here is an interesting thread concerning this topic [Link](https://stackoverflow.com/questions/43162506/undefinedmetricwarning-f-score-is-ill-defined-and-being-set-to-0-0-in-labels-wi)

### 6. Improve your model
Use grid search to find better parameters. 

In [69]:
# Let's take a look at the parameters first
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f80878152f0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [70]:
# parameters for grid search and cross-validation using GridSearchCV
from sklearn.model_selection import GridSearchCV
parameters = {'clf__estimator__n_estimators' : [50, 100]}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=1)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [71]:
cv.fit(X_train, Y_train)
Y_pred = cv.predict(X_test)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 27.2min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [74]:
Y_pred = cv.predict(X_test)
model_testing(Y_test, Y_pred)

related              precision    recall  f1-score   support

          0       0.73      0.28      0.40      1532
          1       0.81      0.97      0.88      5022

avg / total       0.79      0.81      0.77      6554

request              precision    recall  f1-score   support

          0       0.89      0.99      0.94      5390
          1       0.91      0.44      0.59      1164

avg / total       0.89      0.89      0.88      6554

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6527
          1       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.76      0.88      0.82      3814
          1       0.79      0.62      0.69      2740

avg / total       0.77      0.77      0.77      6554

medical_help              precision    recall  f1-score   support

          0       0.93      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Remember that we have left out the child_alone variable?
We left it out, because it would always show 100% accuracy here, as all values were the same (0)

In [77]:
accuracy = (Y_pred == Y_test).mean()
accuracy

related                   0.806836
request                   0.892280
offer                     0.995880
aid_related               0.771895
medical_help              0.924474
medical_products          0.953311
search_and_rescue         0.971773
security                  0.982606
military                  0.966433
water                     0.953006
food                      0.928135
shelter                   0.931950
clothing                  0.983216
money                     0.978944
missing_people            0.988557
refugees                  0.965212
death                     0.957888
other_aid                 0.867714
infrastructure_related    0.933628
transport                 0.953616
buildings                 0.951938
electricity               0.980928
tools                     0.993439
hospitals                 0.987946
shops                     0.995575
aid_centers               0.987031
other_infrastructure      0.955447
weather_related           0.868325
floods              

### 8. Export your model as a pickle file

In [ ]:
import pickle

filename = 'desaster_response_model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 9. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.